In [13]:
import pandas as pd
import altair as alt

In [14]:
# Load data
df = pd.read_csv('dataset.csv')

# Filter for Elvis Presley
elvis_df = df[df['artists'].str.contains('Elvis Presley', case=False, na=False)].copy()

# Convert duration
elvis_df['duration_min'] = elvis_df['duration_ms'] / 60000

# Normalize loudness (scale clarity)
elvis_df['loudness_norm'] = (elvis_df['loudness'] - elvis_df['loudness'].min()) / \
                            (elvis_df['loudness'].max() - elvis_df['loudness'].min())

# Create a single-point selection for interactivity ---
click = alt.selection_point(on='click', fields=['track_name'], empty='none')

elvis_df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_min,loudness_norm
90000,90000,44AyOl4qVkzS48vBsbNXaC,Elvis Presley,Blue Hawaii,Can't Help Falling in Love,80,182360,False,0.396,0.293,...,0.0275,0.9410,0.000196,0.1050,0.343,100.307,3,rock-n-roll,3.039333,0.419858
90004,90004,452KBpASS5lZLeJWX9Ixub,Elvis Presley,Platinum - A Life In Music,Jailhouse Rock,72,146933,False,0.623,0.537,...,0.0726,0.3320,0.000014,0.0963,0.905,166.885,4,rock-n-roll,2.448883,0.656141
90005,90005,1H5IfYyIIAlgDX8zguUzns,Elvis Presley,From Elvis in Memphis,Suspicious Minds,76,261279,False,0.487,0.382,...,0.0309,0.0422,0.000005,0.4110,0.714,116.557,4,rock-n-roll,4.354650,0.605067
90007,90007,3KJombE6eRPrdrhNJza7T8,Elvis Presley,Santa Claus Is Coming To Town,Blue Christmas,0,128080,False,0.506,0.475,...,0.0324,0.8300,0.004110,0.1240,0.596,94.494,4,rock-n-roll,2.134667,0.799848
90050,90050,0D1pEisM3QkiacGXJe5dmd,Elvis Presley,Elvis 30 #1 Hits,(You're The) Devil in Disguise,67,140426,False,0.481,0.733,...,0.1650,0.5750,0.000012,0.1080,0.874,122.909,4,rock-n-roll,2.340433,0.795120


In [15]:
# Valence vs Acousticness: emotional tone vs. instrumentation
points_valence = (
    alt.Chart(elvis_df)
    .mark_circle(size=90, opacity=0.7)
    .encode(
        x=alt.X('acousticness', title='Acousticness (Instrumental Quality)'),
        y=alt.Y('valence', title='Valence (Happiness)'),
        color=alt.condition(click, alt.value('#e41a1c'), alt.Color('tempo:Q', scale=alt.Scale(scheme='plasma'))),
        tooltip=['track_name', 'album_name', 'tempo', 'loudness', 'duration_min']
    )
    .add_params(click)
    .properties(
        title='Elvis Presley Songs: Valence vs. Acousticness (Click to Reveal Song)',
        width=600,
        height=400
    )
)

text_valence = (
    alt.Chart(elvis_df)
    .mark_text(align='left', dx=10, dy=-10, color='black', fontSize=13, fontWeight='bold')
    .encode(
        text=alt.condition(click, 'track_name:N', alt.value('')),
    )
    .transform_filter(click)
)

valence_chart = points_valence + text_valence

In [16]:
# Tempo Distribution: how fast Elvis’ songs are
tempo_hist = (
    alt.Chart(elvis_df)
    .mark_bar(opacity=0.8)
    .encode(
        x=alt.X('tempo:Q', bin=alt.Bin(maxbins=25), title='Tempo (BPM)'),
        y=alt.Y('count()', title='Number of Songs'),
        color=alt.value('#2b83ba')
    )
    .properties(
        title='Distribution of Elvis Presley Song Tempos',
        width=600,
        height=300
    )
)

In [17]:
# Loudness vs Duration: how production intensity relates to song length
points_loudness = (
    alt.Chart(elvis_df)
    .mark_circle(size=90, opacity=0.7, color='#d7191c')
    .encode(
        x=alt.X('duration_min', title='Song Duration (minutes)'),
        y=alt.Y('loudness', title='Loudness (dB)'),
        tooltip=['track_name', 'album_name', 'tempo', 'valence']
    )
    .add_params(click)
    .properties(
        title='Loudness vs Duration (Click a Song)',
        width=600,
        height=400
    )
)

text_loudness = (
    alt.Chart(elvis_df)
    .mark_text(align='left', dx=10, dy=-10, color='black', fontSize=13, fontWeight='bold')
    .encode(
        text=alt.condition(click, 'track_name:N', alt.value('')),
    )
    .transform_filter(click)
)

loudness_chart = points_loudness + text_loudness


In [18]:
# Combine all visuals into one dashboard
final_viz = valence_acoustic & tempo_hist & loudness_duration
final_viz

alt.VConcatChart(...)